In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [2]:
# Insert your Spotify username and the credentials that you obtained from spotify developer
cid = 'XXXXXXX'
secret = 'XXXXXXX'
redirect_uri='http://localhost:7777/callback'
username = 'XXXXXXXX'

In [3]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
# Getting top 1000 tracks from user
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [5]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Feels Like Death,Feels Like Death,03vMyCyCK7pVWjC1i1zur0,Levi Carter,237505,44
1,Deep End Freestyle,Sleepy Hallow Presents: Sleepy For President,2AlYncTpVHKwHb55F9lF6O,Sleepy Hallow,115200,75
2,Glory Boy,Freewave 3,5LpnrXjrt0BOU0iOGH78UN,LUCKI,111048,40
3,Place,Whole Lotta Red,1Bg2CNZw6S4e9cGWPmi0uI,Playboi Carti,117239,69
4,Faith,Faith,0TqNfrOY2IrpFRI2zxsMq4,LUCKI,107467,36


In [6]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [7]:
df_fav = fetch_audio_features(sp, df_favourite)
df_fav.head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
track_name,,,,,,,,,,,,,
Feels Like Death,03vMyCyCK7pVWjC1i1zur0,0.755,0.337000,0.626,110.088,0.0000,-6.247,0.1370,237505,6,0.465,0.293,1
Deep End Freestyle,2AlYncTpVHKwHb55F9lF6O,0.868,0.703000,0.479,133.928,0.0000,-6.687,0.1020,115200,4,0.776,0.445,0
Glory Boy,5LpnrXjrt0BOU0iOGH78UN,0.784,0.096600,0.519,129.953,0.0109,-9.868,0.0748,111048,1,0.599,0.096,1
Place,1Bg2CNZw6S4e9cGWPmi0uI,0.891,0.000674,0.628,131.060,0.0000,-6.350,0.1190,117240,1,0.040,0.105,1
Faith,0TqNfrOY2IrpFRI2zxsMq4,0.878,0.123000,0.584,125.069,0.0000,-10.014,0.1050,107468,10,0.488,0.419,1


In [8]:
# Function to create a df of featured playlist
def featured_playlists(sp):
    id = []
    name = []
    num_tracks = []
 # For looping through the API request  
    response = sp.featured_playlists(limit = 50)
    playlists = response['playlists']
    for i, items in enumerate(playlists['items']):
        id.append(items['id'])
        name.append(items['name'])
        num_tracks.append(items['tracks']['total'])

# Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [9]:
df_playlists = featured_playlists(sp)
df_playlists

,id,name,#tracks
0,37i9dQZF1DX4pUKG1kS0Ac,Guilty Pleasures,151
1,37i9dQZF1DXbYM3nMM0oPk,Mega Hit Mix,75
2,37i9dQZF1DWWBHeXOYZf74,POLLEN,146
3,37i9dQZF1DWXRqgorJj26U,Rock Classics,145
4,37i9dQZF1DX4bSrsRWE9cd,Bliss,74
5,37i9dQZF1DWUa8ZRTfalHk,Pop Rising,85
6,37i9dQZF1DX5Ejj0EkURtP,All Out 10s,150
7,37i9dQZF1DWTcqUzwhNmKv,Kickass Metal,60
8,37i9dQZF1DXdPIbPNr916x,Easy Summer,50
9,37i9dQZF1DX9XIFQuFvzM4,Feelin' Good,100


In [10]:
# Getting the tracks in each playlist
def fetch_playlist_tracks(sp, playlistsid): 
    offset = 0
    tracks = []
    # Make the API request
    while True:
            content = sp.playlist_tracks( playlistsid, fields=None, limit=100, offset=offset, market=None)
            tracks += content['items']
        
            if content['next'] is not None:
                offset += 100
            else:
                break
    
    track_id = []
    track_name = []
    
    for track in tracks:
        track_id.append(track['track']['id'])
        track_name.append(track['track']['name'])
    
# Create the final df
    df_playlists_tracks = pd.DataFrame({"track_id":track_id, "track_name": track_name})
    return df_playlists_tracks

In [11]:
fetch_playlist_tracks(sp,'37i9dQZF1DXaz7CcPuNVXF').head()

,track_id,track_name
0,7vFv0yFGMJW3qVXbAd9BK9,Your Body Is a Wonderland
1,24YvUQnuPL8gObCfSnAobH,"Hey, Soul Sister"
2,3rKYiySCDMUKTw5kGVVhaa,Marvin Gaye (feat. Meghan Trainor)
3,0ULAOsJR33XQzlZVZvjGpB,Don't Dream It's Over
4,0IktbUcnAGrvD03AWnz3Q8,Lucky


In [12]:
# Function to fetch the audio features from the songs of given playlist
def fetch_audio_features(sp, playlist_id):
    playlist = fetch_playlist_tracks(sp, playlist_id)
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key',
                                                             'valence', 'speechiness'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features


In [13]:
# Build the dtaframe froms the playlists
for i, playlist in enumerate(df_playlists['id']):
    try:
        string_command = "df_{} = fetch_audio_features(sp, playlist)".format(playlist)
        print("Create {}".format(string_command))
        exec(string_command)
    except:
        print("playlist with id {} is not valid, skiping ".format(playlist))
        pass


Create df_37i9dQZF1DX4pUKG1kS0Ac = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DXbYM3nMM0oPk = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWWBHeXOYZf74 = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWXRqgorJj26U = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX4bSrsRWE9cd = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWUa8ZRTfalHk = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX5Ejj0EkURtP = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWTcqUzwhNmKv = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DXdPIbPNr916x = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DX9XIFQuFvzM4 = fetch_audio_features(sp, playlist)
Create df_37i9dQZF1DWYAcBZSAVhlf = fetch_audio_features(sp, playlist)


In [14]:
# To understand the playlist, it would be useful to get the average value for each audio feature
def fetch_audio_features_mean(sp, playlist_id):
    Playlist = fetch_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [15]:
df = fetch_audio_features_mean(sp, '37i9dQZF1DX0MLFaUdXnjA')
df.head()

,37i9dQZF1DX0MLFaUdXnjA
danceability,0.605716
acousticness,0.459792
energy,0.465119
tempo,117.807000
instrumentalness,0.005687


In [16]:
# Merge all playlists together to get a large df of tracks
dataframes = []
# Loop through the filenames to populate dataframes with different dataframes 
for i in df_playlists['id']:
    try:
        dataframes.append(fetch_audio_features_mean(sp, i))
    except:
        pass

In [17]:
# Preparing dataset for model creation
X = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), dataframes)
X.head()

,37i9dQZF1DX4pUKG1kS0Ac,37i9dQZF1DXbYM3nMM0oPk,37i9dQZF1DWWBHeXOYZf74,37i9dQZF1DWXRqgorJj26U,37i9dQZF1DX4bSrsRWE9cd,37i9dQZF1DWUa8ZRTfalHk,37i9dQZF1DX5Ejj0EkURtP,37i9dQZF1DWTcqUzwhNmKv,37i9dQZF1DXdPIbPNr916x,37i9dQZF1DX9XIFQuFvzM4,37i9dQZF1DWYAcBZSAVhlf
danceability,0.643397,0.733000,0.666877,0.546407,0.493041,0.635706,0.682853,0.420967,0.630100,0.661030,0.735875
acousticness,0.106590,0.184382,0.289130,0.191717,0.816104,0.243227,0.169380,0.001122,0.236338,0.352801,0.156735
energy,0.753497,0.599280,0.585945,0.720138,0.236964,0.624812,0.666100,0.933650,0.637880,0.593220,0.762463
tempo,124.272464,123.557480,116.746000,122.289014,117.788365,126.002459,118.251593,129.538450,115.934920,117.504610,120.065550
instrumentalness,0.014074,0.013570,0.135805,0.046154,0.164272,0.008420,0.002196,0.082765,0.003852,0.036347,0.045311


In [18]:
Y = pd.DataFrame(df_fav.median(), columns= ['fav_playlist'])
Y = Y.drop('mode')
Y.head()

,fav_playlist
danceability,0.772000
acousticness,0.084300
energy,0.481500
tempo,132.343500
instrumentalness,0.000003


## Model Building

In [19]:
# Analyze feature importances
from sklearn.ensemble import RandomForestRegressor
# Can combine step above with this
forest = RandomForestRegressor(random_state=42, max_depth=5, max_features=10) 
forest.fit(X,Y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature rankings
print("Playlist ranking:")
  
for f in range(len(importances)):
    print("%d. %s %f " % (f + 1, 
            X.columns[f], 
            importances[indices[f]]))

Playlist ranking:
1. 37i9dQZF1DX4pUKG1kS0Ac 0.128664 
2. 37i9dQZF1DXbYM3nMM0oPk 0.119337 
3. 37i9dQZF1DWWBHeXOYZf74 0.108996 
4. 37i9dQZF1DWXRqgorJj26U 0.108948 
5. 37i9dQZF1DX4bSrsRWE9cd 0.092113 
6. 37i9dQZF1DWUa8ZRTfalHk 0.091222 
7. 37i9dQZF1DX5Ejj0EkURtP 0.089925 
8. 37i9dQZF1DWTcqUzwhNmKv 0.079810 
9. 37i9dQZF1DXdPIbPNr916x 0.070683 
10. 37i9dQZF1DX9XIFQuFvzM4 0.070003 
11. 37i9dQZF1DWYAcBZSAVhlf 0.040298 
<ipython-input-19-d8f51d89f499>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X,Y)
